In [1]:
import json
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI()
BASE_URL = "https://nomad-movies.nomadcoders.workers.dev"

In [2]:
# 실제 API 호출 함수들

def get_popular_movies():
    res = requests.get(f"{BASE_URL}/movies")
    return res.json()

def get_movie_details(movie_id: int):
    res = requests.get(f"{BASE_URL}/movies/{movie_id}")
    return res.json()

def get_movie_credits(movie_id: int):
    res = requests.get(f"{BASE_URL}/movies/{movie_id}/credits")
    return res.json()

In [3]:
# OpenAI에게 알려줄 함수(tool) 스키마 정의

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "Get the list of currently popular movies.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "Get detailed information about a specific movie by its ID. Returns title, overview, release date, genres, runtime, etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "Get the cast and crew of a specific movie by its ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "The ID of the movie.",
                    }
                },
                "required": ["movie_id"],
            },
        },
    },
]

# 함수 이름 -> 실제 함수 매핑
available_functions = {
    "get_popular_movies": get_popular_movies,
    "get_movie_details": get_movie_details,
    "get_movie_credits": get_movie_credits,
}

In [4]:
def run_agent(user_message: str):
    messages = [
        {
            "role": "system",
            "content": (
                "You are a movie expert assistant. "
                "Use the provided functions to look up movie information and answer the user's questions. "
                "Always respond in Korean."
            ),
        },
        {"role": "user", "content": user_message},
    ]

    # 에이전트 루프: tool_call이 없을 때까지 반복
    while True:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
        )
        msg = response.choices[0].message
        messages.append(msg)

        # tool call이 없으면 최종 응답
        if not msg.tool_calls:
            print(f"\nAssistant: {msg.content}")
            return msg.content

        # tool call 처리
        for tool_call in msg.tool_calls:
            fn_name = tool_call.function.name
            fn_args = json.loads(tool_call.function.arguments)
            print(f"\n-> Calling {fn_name}({fn_args})")

            result = available_functions[fn_name](**fn_args)

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result, ensure_ascii=False),
                }
            )